In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [32]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Normalization
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import copy



In [ ]:
class EmbeddingGenerator(BaseEstimator, TransformerMixin):
    def __init__(self, epochs=5, batch_size=128, embedding_size=None):
        self.model = None
        self.catagorical_columns = []
        self.numberical_columns = []
        self.epochs = epochs
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.oe = OrdinalEncoder(categories="auto", handle_unknown="use_encoded_value", unknown_value=9999, encoded_missing_value=9999, dtype=np.int32)

    def fit(self, X_train, y_train):
        self.X_train = X_train

    def _get_encoded_categorical_data(self, X, case='train'):
        self.cateegorical_columns = X.select_dtype(include=['object', 'category']).columns.tolist()
        self.n_categories = [X[c].unique() for c in self.cateegorical_columns] #how many level in each categorical column
        if case == 'train':
            encoded_cat_data =





In [10]:
train_df = pd.read_csv("train.csv", low_memory=False)
test_df = pd.read_csv("test.csv", low_memory=False)
store_df = pd.read_csv("store.csv", low_memory=False)

train_data = pd.merge(train_df, store_df, how='inner', on='Store')
test_data = pd.merge(test_df, store_df, how='inner', on='Store')

In [15]:
X_train, y_train =train_data.drop(['Sales'],axis=1), train_data['Sales']
col_to_use = ['Store', 'Open', 'StoreType', 'DayOfWeek', 'Assortment', 'SchoolHoliday']
print(X_train[col_to_use].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   Open           1017209 non-null  int64 
 2   StoreType      1017209 non-null  object
 3   DayOfWeek      1017209 non-null  int64 
 4   Assortment     1017209 non-null  object
 5   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 46.6+ MB
None


In [28]:

cl = X_train.select_dtypes(include=['object','category']).columns.tolist()
[X_train[c].nunique() for c in cl]

[942, 4, 4, 3, 3]

In [39]:
oe = OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value=9999,encoded_missing_value=9999,dtype=np.int32)
oe.fit_transform(X_train[cl])

array([[ 941,    0,    2,    0, 9999],
       [ 941,    0,    0,    0,    1],
       [ 941,    0,    0,    0,    1],
       ...,
       [   0,    1,    0,    2, 9999],
       [   0,    1,    0,    2, 9999],
       [   0,    1,    3,    2,    2]], dtype=int32)

,Date,StateHoliday,StoreType,Assortment,PromoInterval
0,2015-07-31,0,c,a,NaN
1,2015-07-31,0,a,a,"Jan,Apr,Jul,Oct"
2,2015-07-31,0,a,a,"Jan,Apr,Jul,Oct"
3,2015-07-31,0,c,c,NaN
4,2015-07-31,0,a,a,NaN
...,...,...,...,...,...
1017204,2013-01-01,a,a,a,"Jan,Apr,Jul,Oct"
1017205,2013-01-01,a,c,c,NaN
1017206,2013-01-01,a,a,c,NaN
1017207,2013-01-01,a,a,c,NaN
